In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data

In [20]:
from torch import cuda

if cuda.is_available():
    device = 'cuda'
    torch.cuda.manual_seed_all(seed)
    print("running on GPU:", torch.cuda.get_device_name(0))
else:
    device = 'cpu'
    print("running on CPU")

running on CPU


In [3]:
from CommentsDatasets import * # torch dataset setup
from models import * # all our LSTM based models

In [4]:
train_comment_array = torch.load('./StoredTensors/w2v_train_comment_array.pt')
train_title_array = torch.load('./StoredTensors/w2v_train_title_array.pt')
train_labels = torch.load('./StoredTensors/w2v_train_labels.pt')


In [5]:
print(train_comment_array.dtype)
print(type(train_comment_array))
print(train_title_array.dtype)
print(type(train_title_array))

float32
<class 'numpy.ndarray'>
float32
<class 'numpy.ndarray'>


In [6]:
"""
input data should be: (batch_size, max_length, embed_dim)
labels should be: (batch_size,)
"""
print(train_comment_array.shape, train_title_array.shape, train_labels.shape)
print(test_comment_array.shape, test_title_array.shape, test_labels.shape)

(1528, 244, 300) (1528, 13, 300) (1528,)
(102, 126, 300) (102, 19, 300) (102,)


In [26]:
from tqdm import tqdm

def train(training_dataset, n_epochs, batch_size, device, modeltype, model = None, embed_dim = 300, bidi = False, attention = False):
    if model is None:
        """
        embed_dim = 768 for BERT embeddings
        embed_dim = 300 for Word2Vec embeddings 
        """
        model = modeltype(embed_dim = embed_dim, bidi=bidi, attention=attention).to(device)
    else:
        model = model.to(device)
        
    opt = optim.Adam(model.parameters(), lr=0.001)

    loader = torch_data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

    loss_fn = nn.BCELoss()

    losses = []
    for epoch in tqdm(range(n_epochs)):
        epoch_loss = 0
        for comment, comment_pm, label in loader:
            comment = comment.to(device)
            label = label.to(device).type(torch.float32)

            preds = model.forward(comment)
            print(comment.shape, label.shape)
            print(preds.shape)
            
            print("label = ")
            print(label)
            print("Preds = ")
            print(preds)
            
            return model, losses

            opt.zero_grad()
            loss = loss_fn(preds, label)
            loss.backward()
            opt.step()

            epoch_loss += loss.item()
        print('Loss:', epoch_loss)
        losses.append(epoch_loss)


    return model,losses

In [27]:
num_epochs = 5 #running on colab, use 30-40ish
training_data = GeneralDataset(train_comment_array, torch.zeros(1528, 244), train_labels)
trained_model = train(training_data, num_epochs, 16, device, Full_LSTM_Model)

LSTM with bidirectional = False, attention = False


  0%|                                                     | 0/5 [00:00<?, ?it/s]

torch.Size([16, 244, 300]) torch.Size([16])
torch.Size([16])
label = 
tensor([1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.])
Preds = 
tensor([0.6473, 0.6272, 0.6490, 0.6323, 0.6368, 0.6406, 0.6322, 0.6364, 0.6157,
        0.6220, 0.6303, 0.6423, 0.6501, 0.6507, 0.7809, 0.6376],
       grad_fn=<SigmoidBackward0>)


In [33]:
import numpy as np
preds = np.random.rand(10,2)
print(preds)

[[0.94724951 0.44002406]
 [0.77703905 0.94997337]
 [0.52749621 0.67021418]
 [0.60359899 0.52181265]
 [0.30234545 0.07027866]
 [0.20096237 0.82034163]
 [0.68728773 0.10254108]
 [0.23066006 0.24203252]
 [0.57687544 0.49158575]
 [0.22550486 0.92975014]]


In [34]:
pred_flat = np.argmax(preds, axis=1)
print(pred_flat)

[0 1 1 0 0 1 0 1 0 1]


In [35]:
pred_flat = pred_flat.flatten()
print(pred_flat)

[0 1 1 0 0 1 0 1 0 1]


In [36]:
print(pred_flat.shape)

(10,)
